In [ ]:
# nt 現在の人口
# K キャパ
# r 成長率
deriv_Nt(nt, K, r) = r*nt*(1-nt/K)

deriv_Nt2(nt) = r*nt*(1-nt/K)
# function deriv_Nt(nt::Float64, K::Float64, r::Float64)::Float64
#     return r*nt*(1-nt/K)
# end 

In [ ]:
# マルチプルディスパッチ
methods(deriv_Nt)

In [ ]:
# オイラー方程式での近似
next_Nt(nt, K, r, h) = nt + h*deriv_Nt(nt, K, r)

In [ ]:
# 各種パラメータを定義
tmax = 10.0 # 終了時刻
r = 1.0 # パラメータ1
K = 10.0 #パラメータ2
N0 = 1.0 # 人口の初期値
numtimes  = 100 # 0 ≦ t < tmaxの分割数

# ローカル変数にしか型は指定できない
h = tmax/numtimes

In [ ]:
deriv_Nt(0.0, K, r)
# @code_warntype deriv_Nt2(N0)
# @code_warntype next_Nt(N0, K, r, h)
@show next_Nt(N0, K, r, h)
;

In [ ]:
# undef: 初期値指定しない
results = Vector{Float64}(undef, numtimes+1) 
# results2 = zeros(Float64, numtimes+1) 


results[1] = N0


for t in 1:numtimes
    results[t+1] = next_Nt(results[t], K, r, h)
end

In [ ]:
results
;

In [ ]:
using Plots

# = linspace, 等間隔のメッシュ
# 最小値, 最大値, ステップ幅だけメモリに記憶している
# 配列化するには collect(LinRange(XXXXXX))
times = LinRange(0, tmax, numtimes+1)

plot(times, results, label="Numerical", xlabel="time")

In [ ]:
# exact_Nt(N0, K, r, time) =K/(1+(K-N0) / N0 * exp(-r*time))

exact_Nt(time) =K/(1+(K-N0) / N0 * exp(-r*time))

In [ ]:
exact_Nt.(times)
;

In [ ]:
err = abs.(results .- exact_Nt.(times))

# p: プロットオブジェクト
p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
plot!(p, times, results, label="Numerical")
plot!(p, times, err, marker=:x, label="Error")

In [ ]:
# :x  シンボル
# typeof(:x) 文字列
# typeof("x")

# メモリとしては違う????
# a = "x" 
# b = "x" 
a = :x

objectid(a)  # ポインター

In [ ]:
b = :x 

In [ ]:
objectid(b)

In [ ]:
using DifferentialEquations

# ロジスティック方程式の定義
function logistic!(du, u, params, t)
    r, K = params
    du[1] = r * u[1] * (1 - u[1]/K)
end

# パラメータの設定
params = (r, K)

# 初期条件の設定
u0 = [1.0]  # 開始時点の人口サイズ

# 時間範囲の設定
tspan = (0.0, tmax)

# 問題の設定
prob = ODEProblem(logistic!, u0, tspan, params)

# 微分方程式の解 (5次のルンゲクッタ法)
# Tsit5: ソルバー（アルゴリズム）
sol = solve(prob, Tsit5(), abstol=1e-8, reltol=1e-8)
;

In [ ]:
results_de = [u_[1] for u_ in sol.u]

p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
plot!(p, sol.t, results_de, marker=:o)
plot!(p, sol.t, abs.(results_de .- exact_Nt.(sol.t)), label="error")